# NJ Housing Data Analysis

### Goal is to prepare:
* Owner occupied housing data
* Tenant occupied housing data
* County Town level house count and average property tax data
* County Town Zip level house count and average property tax data  

In [7]:
#Import Libraries
import pandas as pd
import json
import requests
import os

In [439]:
#File paths
#url="https://data.nj.gov/api/views/INLINE/rows.json?accessType=DOWNLOAD"
target_path = os.path.join("","rawdhousingdata","Property_Tax_Data.plk")
input_file_path=os.path.join("..","Final Output Data","CountyTownZip_Data.csv")
input_zip_path = os.path.join("","rawdhousingdata","zip_codes.csv")

#Files for plottinf and further analysis
OwnerOccupiedProperty_path=os.path.join("..","Final Output Data","OwnerOccupiedProperty_Data.plk")
TenantOccupiedProperty_path=os.path.join("..","Final Output Data","TenantOccupied_Data.plk")
CountyTownAnalysis_path=os.path.join("..","Final Output Data","CountyTownAnalysis_Data.csv")
CountyTownZipAnalysis_path=os.path.join("..","Final Output Data","CountyTownZipAnalysis_Data.csv")

In [10]:
#Read pickle file
df=pd.read_pickle(target_path)

In [11]:
#Check if data frame is loaded properly
df.head(5)

,TAX YEAR,OWNER NAME,OWNER ADDRESS LINE 1,OWNER ADDRESS LINE 2,PROPERTY LOCATION,MUNICIPALITY,COUNTY,BLOCK,LOT,QUALIFIER,PROPERTY CLASS,LAND DESCRIPTION,BUILDING DESCRIPTION,ACREAGE,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT
0,2019,"KURANT, MARTIN J",206 LAWRENCE AVENUE,LAKEWOOD NJ 08701,206 LAWRENCE AVE,LAKEWOOD TWP,OCEAN,167,10.01,-,RESIDENTIAL,53X100,2SF 2436,.1217,231000,228000,459000,9629.82
1,2019,"HAVAS, JOHN M. & STELLA",35 PARK EDGE,"BERKELEY HEIGHTS, NJ 07922",35 PARK EDGE,BERKELEY HEIGHTS TWP,UNION,301.17,1,CONDO,RESIDENTIAL,-,CONDO UNIT,0,60000,109700,169700,6940.73
2,2019,"SAMUEL, ELEANOR",104 ELLERY AVE,"VILLAS, NJ 08251",104 ELLERY ROAD,LOWER TWP,CAPE MAY,266.04,4,-,RESIDENTIAL,55X100,1S-F-R,.1263,72000,78000,150000,2385.50
3,2019,"ALVAREZ, MARCOS & DIANA",1023 MAPLE AVE,"RIDGEFIELD, NJ 07657",1023 MAPLE AVE,RIDGEFIELD BORO,BERGEN,401,5,-,RESIDENTIAL,100X137,2.5SF1G,.3145,246500,212000,458500,11081.95
4,2019,RYAN KATHERINE,4 VILLA CT,PARK RIDGE NJ 076561824,4 VILLA COURT,PARK RIDGE BORO,BERGEN,503,2,-,RESIDENTIAL,137X95 1.0LT,1SF1G,.2988,241900,140800,382700,10589.31


In [12]:
#Get row and column count
df.shape

(3168402, 18)

In [13]:
#Drop Columns that are not important for analysis
df2=df.drop(["QUALIFIER","OWNER NAME","BLOCK","LOT","LAND DESCRIPTION","BUILDING DESCRIPTION","ACREAGE"],axis=1)

In [15]:
#Keep only residential properties
df3=df2[df2["PROPERTY CLASS"]=="RESIDENTIAL"]

In [16]:
#Display sample data
df3.head()

,TAX YEAR,OWNER ADDRESS LINE 1,OWNER ADDRESS LINE 2,PROPERTY LOCATION,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT
0,2019,206 LAWRENCE AVENUE,LAKEWOOD NJ 08701,206 LAWRENCE AVE,LAKEWOOD TWP,OCEAN,RESIDENTIAL,231000,228000,459000,9629.82
1,2019,35 PARK EDGE,"BERKELEY HEIGHTS, NJ 07922",35 PARK EDGE,BERKELEY HEIGHTS TWP,UNION,RESIDENTIAL,60000,109700,169700,6940.73
2,2019,104 ELLERY AVE,"VILLAS, NJ 08251",104 ELLERY ROAD,LOWER TWP,CAPE MAY,RESIDENTIAL,72000,78000,150000,2385.50
3,2019,1023 MAPLE AVE,"RIDGEFIELD, NJ 07657",1023 MAPLE AVE,RIDGEFIELD BORO,BERGEN,RESIDENTIAL,246500,212000,458500,11081.95
4,2019,4 VILLA CT,PARK RIDGE NJ 076561824,4 VILLA COURT,PARK RIDGE BORO,BERGEN,RESIDENTIAL,241900,140800,382700,10589.31


In [344]:
#Residential record count
df3.shape

(2566541, 11)

In [17]:
#Find if Propery location is same as owner location. We can prepare two data sets- 
#1. With zip code (if property location is same as owner address)
#2. if property location is not same as owner address - means rented and zip code need to researchedd

In [18]:
#Let's start with first dataset

In [30]:
#Get good addresses - owner and property location matches
good_address=df3["OWNER ADDRESS LINE 1"]==df3["PROPERTY LOCATION"]
good_addresses =df3[good_address]

In [31]:
#Check sample data
good_addresses.head()

,TAX YEAR,OWNER ADDRESS LINE 1,OWNER ADDRESS LINE 2,PROPERTY LOCATION,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT
1,2019,35 PARK EDGE,"BERKELEY HEIGHTS, NJ 07922",35 PARK EDGE,BERKELEY HEIGHTS TWP,UNION,RESIDENTIAL,60000,109700,169700,6940.73
3,2019,1023 MAPLE AVE,"RIDGEFIELD, NJ 07657",1023 MAPLE AVE,RIDGEFIELD BORO,BERGEN,RESIDENTIAL,246500,212000,458500,11081.95
6,2019,88 ROSS HALL BLVD S,"PISCATAWAY, NJ 08854",88 ROSS HALL BLVD S,PISCATAWAY TWP,MIDDLESEX,RESIDENTIAL,213600,249300,462900,10631.31
8,2019,36 JOHN ST,"EAST HANOVER, NJ 07936",36 JOHN ST,EAST HANOVER TWP,MORRIS,RESIDENTIAL,189100,292200,481300,10949.58
9,2019,47 WAKEFIELD LN,"PISCATAWAY, NJ 08854",47 WAKEFIELD LN,PISCATAWAY TWP,MIDDLESEX,RESIDENTIAL,258500,596700,855200,19134.93


In [32]:
#Extract zip from address line 2
def getZipCode(x):
    y= str(x).split("NJ")
    if len(y)==2:
        return y[1]
    else:
        return 0

In [33]:
#Extract zip codes for good addresses
zip_code= good_addresses["OWNER ADDRESS LINE 2"].apply(getZipCode)

In [34]:
#Add Zip Code column
good_addresses["Zip Code"]=zip_code.str[:6]

C:\Users\anuaj\Anaconda3\envs\PythonData2020_1\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
#Check sample data
good_addresses.head(2)

,TAX YEAR,OWNER ADDRESS LINE 1,OWNER ADDRESS LINE 2,PROPERTY LOCATION,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT,Zip Code
1,2019,35 PARK EDGE,"BERKELEY HEIGHTS, NJ 07922",35 PARK EDGE,BERKELEY HEIGHTS TWP,UNION,RESIDENTIAL,60000,109700,169700,6940.73,07922
3,2019,1023 MAPLE AVE,"RIDGEFIELD, NJ 07657",1023 MAPLE AVE,RIDGEFIELD BORO,BERGEN,RESIDENTIAL,246500,212000,458500,11081.95,07657


In [36]:
#Check how many records are in good order
good_addresses.shape

(1471609, 12)

In [37]:
#Lets work on second data set- zip code missing 

In [38]:
#Properties that does not have matching owner address lines
nonzip_addresses=df3[~good_address]

In [39]:
nonzip_addresses.head(2)

,TAX YEAR,OWNER ADDRESS LINE 1,OWNER ADDRESS LINE 2,PROPERTY LOCATION,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT
0,2019,206 LAWRENCE AVENUE,LAKEWOOD NJ 08701,206 LAWRENCE AVE,LAKEWOOD TWP,OCEAN,RESIDENTIAL,231000,228000,459000,9629.82
2,2019,104 ELLERY AVE,"VILLAS, NJ 08251",104 ELLERY ROAD,LOWER TWP,CAPE MAY,RESIDENTIAL,72000,78000,150000,2385.50


In [40]:
#There is address mismatch like Crt~ Court, Blvd ~ BOLUVED, inclusion of aprt # etc
#Lets apply the rule - if house number and street name (excludes ST,AVE, BLVD etc), property and owner address is same

In [41]:
#Function to parse address line
def partAddress(x):
    y=x.split(" ")
    if len(y)>=2:
        return y[0]+y[1]
    else:
        return 0

In [42]:
#Apply function to rach address column
owner_add_part = nonzip_addresses["OWNER ADDRESS LINE 1"].apply(partAddress)
prop_add_part = nonzip_addresses["PROPERTY LOCATION"].apply(partAddress)

In [43]:
#Get matched address- second cut
matched_address= nonzip_addresses[owner_add_part == prop_add_part]

In [44]:
#Check sample data
matched_address.head(2)

,TAX YEAR,OWNER ADDRESS LINE 1,OWNER ADDRESS LINE 2,PROPERTY LOCATION,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT
0,2019,206 LAWRENCE AVENUE,LAKEWOOD NJ 08701,206 LAWRENCE AVE,LAKEWOOD TWP,OCEAN,RESIDENTIAL,231000,228000,459000,9629.82
2,2019,104 ELLERY AVE,"VILLAS, NJ 08251",104 ELLERY ROAD,LOWER TWP,CAPE MAY,RESIDENTIAL,72000,78000,150000,2385.50


In [46]:
#Populate zip column for 2nd set of good addresses
zip_code= matched_address["OWNER ADDRESS LINE 2"].apply(getZipCode)
matched_address["Zip Code"]=zip_code.str[:6]

C:\Users\anuaj\Anaconda3\envs\PythonData2020_1\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [47]:
#Check sample data to verify zip column
matched_address.head(2)

,TAX YEAR,OWNER ADDRESS LINE 1,OWNER ADDRESS LINE 2,PROPERTY LOCATION,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT,Zip Code
0,2019,206 LAWRENCE AVENUE,LAKEWOOD NJ 08701,206 LAWRENCE AVE,LAKEWOOD TWP,OCEAN,RESIDENTIAL,231000,228000,459000,9629.82,08701
2,2019,104 ELLERY AVE,"VILLAS, NJ 08251",104 ELLERY ROAD,LOWER TWP,CAPE MAY,RESIDENTIAL,72000,78000,150000,2385.50,08251


In [48]:
#Check 2nd cut good addresses
matched_address.shape

(454606, 12)

In [49]:
#Lets merger good and matched address data frames
address_withzip = pd.concat([good_addresses,matched_address])

In [50]:
#Get the good order address count
address_withzip.shape

(1926215, 12)

In [51]:
#Display sample data
address_withzip.head(2)

,TAX YEAR,OWNER ADDRESS LINE 1,OWNER ADDRESS LINE 2,PROPERTY LOCATION,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT,Zip Code
1,2019,35 PARK EDGE,"BERKELEY HEIGHTS, NJ 07922",35 PARK EDGE,BERKELEY HEIGHTS TWP,UNION,RESIDENTIAL,60000,109700,169700,6940.73,07922
3,2019,1023 MAPLE AVE,"RIDGEFIELD, NJ 07657",1023 MAPLE AVE,RIDGEFIELD BORO,BERGEN,RESIDENTIAL,246500,212000,458500,11081.95,07657


In [52]:
#Second part of owner address not matching with property address - possibly rented or investment property

In [56]:
#Rented or investment houses (non owner occupied)
investment_properties= nonzip_addresses[owner_add_part != prop_add_part]

In [57]:
#Get the rented or investment house count
investment_properties.shape

(640326, 11)

In [59]:
#Display sample data
investment_properties.head(2)

,TAX YEAR,OWNER ADDRESS LINE 1,OWNER ADDRESS LINE 2,PROPERTY LOCATION,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT
5,2019,405 NEWBOLD ROAD,JENKINTOWN PA 19046,330 37TH ST SO,BRIGANTINE CITY,ATLANTIC,RESIDENTIAL,249600,223700,473300,8221.22
7,2019,"400 EAST 20TH ST,APT 4B","NEW YORK, NY 100098101",104 WEST BAYBERRY WAY,TOMS RIVER TWP,OCEAN,RESIDENTIAL,146600,61300,207900,4893.96


In [60]:
#Let's get the town-zip data and try to populate the imvestment property zip 

In [245]:
zip_df=pd.read_csv(input_file_path,dtype={'Zip Code':'str'})

In [240]:
zip_df.head(2)

,County,Town,Zip Code,Zip Type
0,Middlesex,Avenel,07001,Standard
1,Hudson,Bayonne,07002,Standard


In [145]:
#Keep only standard zip types
zip_std=zip_df["Zip Type"]=="Standard"
zip_df=zip_df[zip_std]

In [98]:
#Grt city count
len(zip_df)

566

In [117]:
#Display results
investment_properties.head(2)

,TAX YEAR,OWNER ADDRESS LINE 1,OWNER ADDRESS LINE 2,PROPERTY LOCATION,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT
5,2019,405 NEWBOLD ROAD,JENKINTOWN PA 19046,330 37TH ST SO,BRIGANTINE CITY,ATLANTIC,RESIDENTIAL,249600,223700,473300,8221.22
7,2019,"400 EAST 20TH ST,APT 4B","NEW YORK, NY 100098101",104 WEST BAYBERRY WAY,TOMS RIVER TWP,OCEAN,RESIDENTIAL,146600,61300,207900,4893.96


In [93]:
#We wont be able to map 100% zip codes for multi zip code towns

In [175]:
def findZip(x):
    for index,row in zip_df.iterrows():
        izip=(row["Town"]).upper()
        if x.find(izip) != -1:
            return str(row["Zip Code"])

In [177]:
#Get zip code for investment properties
investment_zip=investment_properties["MUNICIPALITY"].apply(findZip)

In [178]:
#Copy investment data set
investment_zip_prop=investment_properties.copy()

In [179]:
#Add Zip code column to investment zip
investment_zip_prop["Zip Code"]=investment_zip

In [180]:
#Display sample results
investment_zip_prop.head(2)

,TAX YEAR,OWNER ADDRESS LINE 1,OWNER ADDRESS LINE 2,PROPERTY LOCATION,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT,Zip Code
5,2019,405 NEWBOLD ROAD,JENKINTOWN PA 19046,330 37TH ST SO,BRIGANTINE CITY,ATLANTIC,RESIDENTIAL,249600,223700,473300,8221.22,08203
7,2019,"400 EAST 20TH ST,APT 4B","NEW YORK, NY 100098101",104 WEST BAYBERRY WAY,TOMS RIVER TWP,OCEAN,RESIDENTIAL,146600,61300,207900,4893.96,08753


In [189]:
investment_zip_prop.tail(2)

,TAX YEAR,OWNER ADDRESS LINE 1,OWNER ADDRESS LINE 2,PROPERTY LOCATION,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT,Zip Code
3168389,2019,88 HUSTED STATION RD,"ELMER, NJ 08318",27 ARAPAHO PL,GALLOWAY TWP,ATLANTIC,RESIDENTIAL,17500,52600,70100,1661.18,None
3168400,2019,97 PLEASENT AVE.,"DUMONT, NJ 07628",97 PLEASANT AVE,DUMONT BORO,BERGEN,RESIDENTIAL,178300,179900,358200,12873.71,07628


In [186]:
investment_zip_prop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 640326 entries, 5 to 3168400
Data columns (total 12 columns):
TAX YEAR                 640326 non-null int64
OWNER ADDRESS LINE 1     640326 non-null object
OWNER ADDRESS LINE 2     640326 non-null object
PROPERTY LOCATION        640326 non-null object
MUNICIPALITY             640326 non-null object
COUNTY                   640326 non-null object
PROPERTY CLASS           640326 non-null object
LAND VALUE               640326 non-null int64
BUILDING VALUE           640326 non-null int64
NET VALUE                640326 non-null int64
PRIOR YEAR TAX AMOUNT    640326 non-null float64
Zip Code                 501096 non-null object
dtypes: float64(1), int64(4), object(7)
memory usage: 63.5+ MB


In [198]:
#Investment properties whhere zip codes are populated (640326-501096=139230)
investment_zip_prop_good=investment_zip_prop[~investment_zip_prop["Zip Code"].isnull()]
investment_zip_prop_good.head(2)

,TAX YEAR,OWNER ADDRESS LINE 1,OWNER ADDRESS LINE 2,PROPERTY LOCATION,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT,Zip Code
5,2019,405 NEWBOLD ROAD,JENKINTOWN PA 19046,330 37TH ST SO,BRIGANTINE CITY,ATLANTIC,RESIDENTIAL,249600,223700,473300,8221.22,08203
7,2019,"400 EAST 20TH ST,APT 4B","NEW YORK, NY 100098101",104 WEST BAYBERRY WAY,TOMS RIVER TWP,OCEAN,RESIDENTIAL,146600,61300,207900,4893.96,08753


In [345]:
#Investment property row count with first zip processing
investment_zip_prop_good.shape

(501096, 12)

In [204]:
#Lets merger good and matched address data frames
address_withzip = pd.concat([address_withzip,investment_zip_prop_good])

In [205]:
#Get record count of addresses with zip code
address_withzip.shape

(2427311, 12)

In [200]:
#Investment properties whete zip codes are not populated
investment_nozip_prop=investment_zip_prop[investment_zip_prop["Zip Code"].isnull()]
investment_nozip_prop.head(2)

,TAX YEAR,OWNER ADDRESS LINE 1,OWNER ADDRESS LINE 2,PROPERTY LOCATION,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT,Zip Code
14,2019,7 CORNELL RD,ATCO NJ 08004,16 THORNTON AVENUE,WINSLOW TWP,CAMDEN,RESIDENTIAL,45400,211800,257200,9097.16,None
31,2019,709 MORGAN AVENUE,PALMYRA NJ 08065,32 KIRK LA,PINE HILL BORO,CAMDEN,RESIDENTIAL,19100,51000,70100,3809.93,None


In [232]:
#Let's get the list of towns
citi_list=investment_nozip_prop["MUNICIPALITY"].tolist()
cities=set(citi_list)
citi_list=list(cities)

In [237]:
#143 towns missing in zip code
citi_list

['WHITE TWP',
 'SURF CITY BORO',
 'NEW HANOVER TWP',
 'INDEPENDENCE TWP',
 'WANTAGE TWP',
 'TAVISTOCK BORO',
 'CHESILHURST BORO',
 'EAST AMWELL TWP',
 'ALPINE BORO',
 'FAIRLAWN BORO',
 'PLUMSTED TWP',
 'FIELDSBORO BORO',
 'MT LAUREL TWP',
 'CINNAMINSON TWP',
 'MONTGOMERY TWP',
 'FRELINGHUYSEN TWP',
 'HI NELLA BORO',
 'HARDWICK TWP',
 'ELSINBORO TWP',
 'PT PLEASANT BEACH BORO',
 'MANNINGTON TWP',
 'EGG HARBOR TWP',
 'WALL TWP',
 'JAMESBURG BORO',
 'HARDING TWP',
 'LONG BEACH TWP',
 'LOGAN TWP',
 'ALPHA BORO',
 'INTERLAKEN BORO',
 'QUINTON TWP',
 'DOWNE TWP',
 'COMMERCIAL TWP',
 'MONROE TWP',
 'MANSFIELD TWP',
 'PINE HILL BORO',
 'MT HOLLY TWP',
 'LOWER TWP',
 'HASBROUCK HGHTS BORO',
 'EASTAMPTON TWP',
 'ALLAMUCHY TWP',
 'EAST WINDSOR TWP',
 'LACEY TWP',
 'HOPE TWP',
 'KINGWOOD TWP',
 'CORBIN CITY CITY',
 'ROCKLEIGH BORO',
 'OCEAN TWP',
 'WALPACK TWP',
 'OLD TAPPAN BORO',
 'LIBERTY TWP',
 'UPPER PITTSGROVE TWP',
 'UPPER SADDLE RIV BORO',
 'GREEN BROOK TWP',
 'ALEXANDRIA TWP',
 'STOW CREE

In [331]:
#Get additional zip code records - Townships that are not part of standard zip code of towns
zip_df1=pd.read_csv(input_zip_path,dtype={'Zip Code':'str'})

In [325]:
#Function to get zip codes
def findZipx(x):
    for index,row in zip_df1.iterrows():
        try:
            izip=str(row["Township"]).split(" ")[0].upper()
        except:
            izip=str(row["Township"]).upper()
        finally:
            pass
        if x.find(izip) != -1:
            return str(row["Zip Code"])

In [332]:
#Get zip code for investment properties
investment_nozips=investment_nozip_prop["MUNICIPALITY"].apply(findZipx)

In [334]:
#Display results
investment_nozip_prop.head(2)

,TAX YEAR,OWNER ADDRESS LINE 1,OWNER ADDRESS LINE 2,PROPERTY LOCATION,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT,Zip Code
14,2019,7 CORNELL RD,ATCO NJ 08004,16 THORNTON AVENUE,WINSLOW TWP,CAMDEN,RESIDENTIAL,45400,211800,257200,9097.16,None
31,2019,709 MORGAN AVENUE,PALMYRA NJ 08065,32 KIRK LA,PINE HILL BORO,CAMDEN,RESIDENTIAL,19100,51000,70100,3809.93,None


In [335]:
#Drop existing zip code column before we append same column
investment_nozip_prop=investment_nozip_prop.drop("Zip Code",axis=1)

In [336]:
#Display results
investment_nozip_prop.head(2)

,TAX YEAR,OWNER ADDRESS LINE 1,OWNER ADDRESS LINE 2,PROPERTY LOCATION,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT
14,2019,7 CORNELL RD,ATCO NJ 08004,16 THORNTON AVENUE,WINSLOW TWP,CAMDEN,RESIDENTIAL,45400,211800,257200,9097.16
31,2019,709 MORGAN AVENUE,PALMYRA NJ 08065,32 KIRK LA,PINE HILL BORO,CAMDEN,RESIDENTIAL,19100,51000,70100,3809.93


In [337]:
#Add Zip code column to investment non zip dataset
investment_nozip_prop["Zip Code"]=investment_nozips

In [338]:
#Display sample results
investment_nozip_prop.head(2)

,TAX YEAR,OWNER ADDRESS LINE 1,OWNER ADDRESS LINE 2,PROPERTY LOCATION,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT,Zip Code
14,2019,7 CORNELL RD,ATCO NJ 08004,16 THORNTON AVENUE,WINSLOW TWP,CAMDEN,RESIDENTIAL,45400,211800,257200,9097.16,08095
31,2019,709 MORGAN AVENUE,PALMYRA NJ 08065,32 KIRK LA,PINE HILL BORO,CAMDEN,RESIDENTIAL,19100,51000,70100,3809.93,08083


In [339]:
#Get the investment property rows that are populated with zip codes
investment_prop_corrected=investment_nozip_prop[~investment_nozip_prop["Zip Code"].isnull()]

In [340]:
#Check record count of zip populated properties
investment_prop_corrected.shape

(139230, 12)

In [341]:
#Lets merger good and matched address data frames
address_withzip = pd.concat([address_withzip,investment_prop_corrected])

In [342]:
#Get record count of addresses with zip code
address_withzip.shape

(2566541, 12)

In [343]:
#2nd round = Investment properties whete zip codes are not populated
investment_nozip_tbd=investment_nozip_prop[investment_nozip_prop["Zip Code"].isnull()]
investment_nozip_tbd.shape

(0, 12)

### End of data cleaning and processing

## Summary:
* Total Available Raw Data Records (df): 3,168,402
* Total Residential property records (df3): 2,566,541
    * Owner Occupied:
        * Exact Address Match (good_addresses): 1,471,609
        * Matched records with street # and name (exclude Dr, Blvd etc):454,606
    * Investment Properties: 
         * Investment 1st round (investment_zip_prop_good): 501,096 
         * Investment 2nd round (investment_prop_corrected): 139,230 #With multiple townships with shared zip code.
* Total processed with zip code population: 2,566,541 
* Investment(Tenant Occupied)/Total Residential=24.94%

In [462]:
#Owner occupied dataset
owner_occupied_address = pd.concat([good_addresses,matched_address])

### Export Owner Occupied Data (with zip code)

In [463]:
#Get the data shape
owner_occupied_address.shape

(1926215, 12)

In [464]:
#Drop columns that are not relvant for further analysis
owner_occupied_address=owner_occupied_address.drop(["OWNER ADDRESS LINE 1","OWNER ADDRESS LINE 2","PROPERTY LOCATION"],axis=1)

In [465]:
#Export data
owner_occupied_address.to_pickle(OwnerOccupiedProperty_path)

### Export Tenant Occupied Data (with zip code)

In [466]:
#Investment Property dataset
tenant_occupied_address = pd.concat([investment_zip_prop_good,investment_prop_corrected])

In [467]:
#Get the data shape
tenant_occupied_address.shape

(640326, 12)

In [468]:
#Drop columns that are not relvant for further analysis
tenant_occupied_address=tenant_occupied_address.drop(["OWNER ADDRESS LINE 1","OWNER ADDRESS LINE 2","PROPERTY LOCATION"],axis=1)

In [469]:
#Export data
tenant_occupied_address.to_pickle(TenantOccupiedProperty_path)

## Property Tax Analysis:  County-Town Level (All Properties)

In [383]:
#Drop columns that are not relvant for further analysis
address_data=address_withzip.drop(["OWNER ADDRESS LINE 1","OWNER ADDRESS LINE 2","PROPERTY LOCATION"],axis=1)

In [384]:
address_data.head(5)

,TAX YEAR,MUNICIPALITY,COUNTY,PROPERTY CLASS,LAND VALUE,BUILDING VALUE,NET VALUE,PRIOR YEAR TAX AMOUNT,Zip Code
1,2019,BERKELEY HEIGHTS TWP,UNION,RESIDENTIAL,60000,109700,169700,6940.73,07922
3,2019,RIDGEFIELD BORO,BERGEN,RESIDENTIAL,246500,212000,458500,11081.95,07657
6,2019,PISCATAWAY TWP,MIDDLESEX,RESIDENTIAL,213600,249300,462900,10631.31,08854
8,2019,EAST HANOVER TWP,MORRIS,RESIDENTIAL,189100,292200,481300,10949.58,07936
9,2019,PISCATAWAY TWP,MIDDLESEX,RESIDENTIAL,258500,596700,855200,19134.93,08854


In [386]:
#At County and Town Level Data analysis

In [387]:
#Group by 
address_data_town=address_data.groupby(["TAX YEAR","COUNTY","MUNICIPALITY","PROPERTY CLASS"])

In [388]:
#Get Count and average for grouped block
address_data_town=address_data_town.agg(["count","mean"])

In [389]:
#Display results
address_data_town.head()

LAND VALUE                 \
                                                         count           mean   
TAX YEAR COUNTY   MUNICIPALITY       PROPERTY CLASS                             
2019     ATLANTIC ABSECON CITY       RESIDENTIAL          3278   73053.965833   
                  ATLANTIC CITY CITY RESIDENTIAL         10825   59312.438799   
                  BRIGANTINE CITY    RESIDENTIAL          8594  227007.609960   
                  BUENA BORO         RESIDENTIAL          1304   56245.782209   
                  BUENA VISTA TWP    RESIDENTIAL          2400   67443.791667   

                                                    BUILDING VALUE  \
                                                             count   
TAX YEAR COUNTY   MUNICIPALITY       PROPERTY CLASS                  
2019     ATLANTIC ABSECON CITY       RESIDENTIAL              3278   
                  ATLANTIC CITY CITY RESIDENTIAL             10825   
                  BRIGANTINE CITY    RESIDENTIAL              8594   
                  BUENA BORO         RESIDENTIAL              1304   
                  BUENA VISTA TWP    RESIDENTIAL              2400   

                                                                   NET VALUE  \
                                                              mean     count   
TAX YEAR COUNTY   MUNICIPALITY       PROPERTY CLASS                            
2019     ATLANTIC ABSECON CITY       RESIDENTIAL      96368.242831      3278   
                  ATLANTIC CITY CITY RESIDENTIAL      61868.032333     10825   
                  BRIGANTINE CITY    RESIDENTIAL     140924.970910      8594   
                  BUENA BORO         RESIDENTIAL     115850.613497      1304   
                  BUENA VISTA TWP    RESIDENTIAL     146290.958333      2400   

                                                                    \
                                                              mean   
TAX YEAR COUNTY   MUNICIPALITY       PROPERTY CLASS                  
2019     ATLANTIC ABSECON CITY       RESIDENTIAL     169422.208664   
                  ATLANTIC CITY CITY RESIDENTIAL     121136.618938   
                  BRIGANTINE CITY    RESIDENTIAL     367932.580870   
                  BUENA BORO         RESIDENTIAL     172096.395706   
                  BUENA VISTA TWP    RESIDENTIAL     213734.750000   

                                                    PRIOR YEAR TAX AMOUNT  \
                                                                    count   
TAX YEAR COUNTY   MUNICIPALITY       PROPERTY CLASS                         
2019     ATLANTIC ABSECON CITY       RESIDENTIAL                     3278   
                  ATLANTIC CITY CITY RESIDENTIAL                    10825   
                  BRIGANTINE CITY    RESIDENTIAL                     8594   
                  BUENA BORO         RESIDENTIAL                     1304   
                  BUENA VISTA TWP    RESIDENTIAL                     2400   

                                                                  
                                                            mean  
TAX YEAR COUNTY   MUNICIPALITY       PROPERTY CLASS               
2019     ATLANTIC ABSECON CITY       RESIDENTIAL     5580.447108  
                  ATLANTIC CITY CITY RESIDENTIAL     4520.335151  
                  BRIGANTINE CITY    RESIDENTIAL     6365.360333  
                  BUENA BORO         RESIDENTIAL     5561.771511  
                  BUENA VISTA TWP    RESIDENTIAL     5640.600588

In [390]:
#Drop multiple count columns and rename land count to # of properties

In [391]:
address_data_town=address_data_town.drop([('BUILDING VALUE', 'count'),('NET VALUE', 'count'),('PRIOR YEAR TAX AMOUNT', 'count')],axis=1)

In [392]:
#Index reset
address_data_town=address_data_town.reset_index()

In [395]:
#Display result
address_data_town.head(2)

TAX YEAR    COUNTY        MUNICIPALITY PROPERTY CLASS LAND VALUE  \
                                                             count   
0     2019  ATLANTIC        ABSECON CITY    RESIDENTIAL       3278   
1     2019  ATLANTIC  ATLANTIC CITY CITY    RESIDENTIAL      10825   

                BUILDING VALUE      NET VALUE PRIOR YEAR TAX AMOUNT  
           mean           mean           mean                  mean  
0  73053.965833   96368.242831  169422.208664           5580.447108  
1  59312.438799   61868.032333  121136.618938           4520.335151

In [396]:
#Rename columns
address_data_town.columns = ['Tax Year', 'County', 'Municipality',"Property Type","Property Count","Avg Land Value","Avg Bldg. Value","Avg Property Value","Prior Year Tax"]

In [399]:
#Format data columns
address_data_town[["Avg Land Value","Avg Bldg. Value","Avg Property Value","Prior Year Tax"]]=address_data_town[["Avg Land Value","Avg Bldg. Value","Avg Property Value","Prior Year Tax"]].applymap("${:.2f}".format)

In [400]:
#Display results
address_data_town.head()

,Tax Year,County,Municipality,Property Type,Property Count,Avg Land Value,Avg Bldg. Value,Avg Property Value,Prior Year Tax
0,2019,ATLANTIC,ABSECON CITY,RESIDENTIAL,3278,$73053.97,$96368.24,$169422.21,$5580.45
1,2019,ATLANTIC,ATLANTIC CITY CITY,RESIDENTIAL,10825,$59312.44,$61868.03,$121136.62,$4520.34
2,2019,ATLANTIC,BRIGANTINE CITY,RESIDENTIAL,8594,$227007.61,$140924.97,$367932.58,$6365.36
3,2019,ATLANTIC,BUENA BORO,RESIDENTIAL,1304,$56245.78,$115850.61,$172096.40,$5561.77
4,2019,ATLANTIC,BUENA VISTA TWP,RESIDENTIAL,2400,$67443.79,$146290.96,$213734.75,$5640.60


In [403]:
#Find row count-- Indicates how many towns are covered under this dayaset
address_data_town.shape

(564, 9)

In [437]:
#Export data
address_data_town.to_csv(CountyTownAnalysis_path)

## Data Analysis: County-Town-Zip Level (All Properties)

In [405]:
#Group by 
address_data=address_data.groupby(["TAX YEAR","COUNTY","MUNICIPALITY","Zip Code","PROPERTY CLASS"])

In [406]:
#Get Count and average for grouped block
address_data=address_data.agg(["count","mean"])

In [407]:
#Display results
address_data.head()

LAND VALUE  \
                                                            count   
TAX YEAR COUNTY   MUNICIPALITY Zip Code PROPERTY CLASS              
2019     ATLANTIC ABSECON CITY  08201   RESIDENTIAL          2470   
                                08203   RESIDENTIAL             4   
                                08204   RESIDENTIAL             1   
                                08205   RESIDENTIAL             4   
                                08207   RESIDENTIAL             1   

                                                                      \
                                                                mean   
TAX YEAR COUNTY   MUNICIPALITY Zip Code PROPERTY CLASS                 
2019     ATLANTIC ABSECON CITY  08201   RESIDENTIAL     76904.251012   
                                08203   RESIDENTIAL     76075.000000   
                                08204   RESIDENTIAL     96800.000000   
                                08205   RESIDENTIAL     69300.000000   
                                08207   RESIDENTIAL     86700.000000   

                                                       BUILDING VALUE  \
                                                                count   
TAX YEAR COUNTY   MUNICIPALITY Zip Code PROPERTY CLASS                  
2019     ATLANTIC ABSECON CITY  08201   RESIDENTIAL              2470   
                                08203   RESIDENTIAL                 4   
                                08204   RESIDENTIAL                 1   
                                08205   RESIDENTIAL                 4   
                                08207   RESIDENTIAL                 1   

                                                                       \
                                                                 mean   
TAX YEAR COUNTY   MUNICIPALITY Zip Code PROPERTY CLASS                  
2019     ATLANTIC ABSECON CITY  08201   RESIDENTIAL     103688.137652   
                                08203   RESIDENTIAL      89775.000000   
                                08204   RESIDENTIAL      52200.000000   
                                08205   RESIDENTIAL     104500.000000   
                                08207   RESIDENTIAL      87300.000000   

                                                       NET VALUE  \
                                                           count   
TAX YEAR COUNTY   MUNICIPALITY Zip Code PROPERTY CLASS             
2019     ATLANTIC ABSECON CITY  08201   RESIDENTIAL         2470   
                                08203   RESIDENTIAL            4   
                                08204   RESIDENTIAL            1   
                                08205   RESIDENTIAL            4   
                                08207   RESIDENTIAL            1   

                                                                       \
                                                                 mean   
TAX YEAR COUNTY   MUNICIPALITY Zip Code PROPERTY CLASS                  
2019     ATLANTIC ABSECON CITY  08201   RESIDENTIAL     180592.388664   
                                08203   RESIDENTIAL     165850.000000   
                                08204   RESIDENTIAL     149000.000000   
                                08205   RESIDENTIAL     173800.000000   
                                08207   RESIDENTIAL     174000.000000   

                                                       PRIOR YEAR TAX AMOUNT  \
                                                                       count   
TAX YEAR COUNTY   MUNICIPALITY Zip Code PROPERTY CLASS                         
2019     ATLANTIC ABSECON CITY  08201   RESIDENTIAL                     2470   
                                08203   RESIDENTIAL                        4   
                                08204   RESIDENTIAL                        1   
                                08205   RESIDENTIAL                        4   
                                08207   RESIDENTIAL              

In [408]:
#Drop multiple count columns and rename land count to # of properties

In [409]:
address_data=address_data.drop([('BUILDING VALUE', 'count'),('NET VALUE', 'count'),('PRIOR YEAR TAX AMOUNT', 'count')],axis=1)

In [410]:
#Display results
address_data.head()

LAND VALUE  \
                                                            count   
TAX YEAR COUNTY   MUNICIPALITY Zip Code PROPERTY CLASS              
2019     ATLANTIC ABSECON CITY  08201   RESIDENTIAL          2470   
                                08203   RESIDENTIAL             4   
                                08204   RESIDENTIAL             1   
                                08205   RESIDENTIAL             4   
                                08207   RESIDENTIAL             1   

                                                                      \
                                                                mean   
TAX YEAR COUNTY   MUNICIPALITY Zip Code PROPERTY CLASS                 
2019     ATLANTIC ABSECON CITY  08201   RESIDENTIAL     76904.251012   
                                08203   RESIDENTIAL     76075.000000   
                                08204   RESIDENTIAL     96800.000000   
                                08205   RESIDENTIAL     69300.000000   
                                08207   RESIDENTIAL     86700.000000   

                                                       BUILDING VALUE  \
                                                                 mean   
TAX YEAR COUNTY   MUNICIPALITY Zip Code PROPERTY CLASS                  
2019     ATLANTIC ABSECON CITY  08201   RESIDENTIAL     103688.137652   
                                08203   RESIDENTIAL      89775.000000   
                                08204   RESIDENTIAL      52200.000000   
                                08205   RESIDENTIAL     104500.000000   
                                08207   RESIDENTIAL      87300.000000   

                                                            NET VALUE  \
                                                                 mean   
TAX YEAR COUNTY   MUNICIPALITY Zip Code PROPERTY CLASS                  
2019     ATLANTIC ABSECON CITY  08201   RESIDENTIAL     180592.388664   
                                08203   RESIDENTIAL     165850.000000   
                                08204   RESIDENTIAL     149000.000000   
                                08205   RESIDENTIAL     173800.000000   
                                08207   RESIDENTIAL     174000.000000   

                                                       PRIOR YEAR TAX AMOUNT  
                                                                        mean  
TAX YEAR COUNTY   MUNICIPALITY Zip Code PROPERTY CLASS                        
2019     ATLANTIC ABSECON CITY  08201   RESIDENTIAL               5935.80387  
                                08203   RESIDENTIAL               5459.78250  
                                08204   RESIDENTIAL               4905.08000  
                                08205   RESIDENTIAL               5658.99500  
                                08207   RESIDENTIAL               5728.08000

In [411]:
#Index reset
address_data=address_data.reset_index()

In [412]:
#Rename columns
address_data.columns = ['Tax Year', 'County', 'Municipality',"Zip Code","Property Type","Property Count","Avg Land Value","Avg Bldg. Value","Avg Property Value","Prior Year Tax"]

In [413]:
#Format data columns
address_data[["Avg Land Value","Avg Bldg. Value","Avg Property Value","Prior Year Tax"]]=address_data[["Avg Land Value","Avg Bldg. Value","Avg Property Value","Prior Year Tax"]].applymap("${:.2f}".format)

In [414]:
#Find row count
address_data.shape

(5594, 10)

In [415]:
#Display results
address_data.head(5)

,Tax Year,County,Municipality,Zip Code,Property Type,Property Count,Avg Land Value,Avg Bldg. Value,Avg Property Value,Prior Year Tax
0,2019,ATLANTIC,ABSECON CITY,08201,RESIDENTIAL,2470,$76904.25,$103688.14,$180592.39,$5935.80
1,2019,ATLANTIC,ABSECON CITY,08203,RESIDENTIAL,4,$76075.00,$89775.00,$165850.00,$5459.78
2,2019,ATLANTIC,ABSECON CITY,08204,RESIDENTIAL,1,$96800.00,$52200.00,$149000.00,$4905.08
3,2019,ATLANTIC,ABSECON CITY,08205,RESIDENTIAL,4,$69300.00,$104500.00,$173800.00,$5658.99
4,2019,ATLANTIC,ABSECON CITY,08207,RESIDENTIAL,1,$86700.00,$87300.00,$174000.00,$5728.08


In [416]:
#Get properties with count>100 () #Assumption: Eliminate zip code that has fewer than 100 houses
count_100 = address_data["Property Count"]>100

In [417]:
#Drop all properties that are insgnificant in that zip code
address_data_analysis=address_data[count_100]

In [418]:
address_data_analysis.head(10)

,Tax Year,County,Municipality,Zip Code,Property Type,Property Count,Avg Land Value,Avg Bldg. Value,Avg Property Value,Prior Year Tax
0,2019,ATLANTIC,ABSECON CITY,08201,RESIDENTIAL,2470,$76904.25,$103688.14,$180592.39,$5935.80
8,2019,ATLANTIC,ABSECON CITY,08201,RESIDENTIAL,792,$61028.79,$73660.98,$134689.77,$4476.00
14,2019,ATLANTIC,ATLANTIC CITY CITY,08401,RESIDENTIAL,3755,$61296.46,$65385.57,$126671.64,$4708.23
20,2019,ATLANTIC,ATLANTIC CITY CITY,08401,RESIDENTIAL,6791,$57860.00,$59687.68,$117483.52,$4406.12
28,2019,ATLANTIC,BRIGANTINE CITY,08203,RESIDENTIAL,2407,$224343.42,$150998.71,$375342.13,$6462.11
33,2019,ATLANTIC,BRIGANTINE CITY,08203,RESIDENTIAL,6169,$228052.81,$136946.73,$364999.55,$6326.93
36,2019,ATLANTIC,BUENA BORO,08326,RESIDENTIAL,179,$55219.55,$122843.02,$178062.57,$5695.14
37,2019,ATLANTIC,BUENA BORO,08341,RESIDENTIAL,153,$51256.21,$120517.65,$171773.86,$5602.31
40,2019,ATLANTIC,BUENA BORO,08360,RESIDENTIAL,139,$64509.35,$120388.49,$184897.84,$5984.70
43,2019,ATLANTIC,BUENA BORO,08310,RESIDENTIAL,646,$55132.35,$109001.24,$164133.59,$5305.30


In [436]:
#Export data
address_data_analysis.to_csv(CountyTownZipAnalysis_path)